In [ ]:
# %load example1a_small
"""small example"""
import sys
sys.path.append("../../..")
import plantbox as pb
import vtk_plot as vp

rs = pb.RootSystem()

# Open plant and root parameter from a file
path = "../../../modelparameter/rootsystem/"
name = "Anagallis_femina_Leitner_2010"
rs.readParameters(path + name + ".xml")

# Initialize
rs.initialize()

# Simulate
rs.simulate(30, True)

# Export final result (as vtp)
rs.write("results/example_1a.vtp")

# Plot, using vtk
vp.plot_roots(rs, "creationTime")


#### Oblibene IPython magics

In [ ]:
%%writefile
%pycat
%load

In [ ]:
%% magicwords help

In [ ]:
%pycat example1_LAB200811_multiple.py

In [ ]:
import example2a_initializeparams

In [ ]:
import example2c_lateralspacing

In [ ]:
import example2d_length

In [ ]:
import example2e_roottips

## WIP

In [92]:
#%%writefile example1_LAB200903_multiple_hydrotropism_centerWORKS.py
"""hydrotropism in a middle of a two plants"""
import sys; sys.path.append("../../..")
import plantbox as pb
import vtk_plot as vp

rs = pb.RootSystem()
path = "../../../modelparameter/rootsystem/"
name = ["PS_Pages_mod1bmodA", "PS_Pages_mod1bmodB", "PS_Pages_mod1bmodB", "PS_Pages_mod1bmodB"]

# Static soil property in a thin layer
maxS = 10  # maximal
minS = 0.1  # minimal
slope = 9  # linear gradient between min and max (cm), half length of linear interpolation between fmax and fmin
box = pb.SDF_PlantBox(10, 10, 10)  # cm
layer = pb.SDF_RotateTranslate(box, pb.Vector3d(0, 0, -13))
soil_prop = pb.SoilLookUpSDF(layer, maxS, minS, slope)

C = 2  # number of columns
R = 1   # and rows
dist = 28  # distance between the root systems [cm]

# Initializes C*R root systems
allRS = []
for i in range(0, C):
    for j in range(0, R):
        rs = pb.RootSystem()
        rs.readParameters(path + name[i] + ".xml")
        
        # Manually set tropism to hydrotropism for the first six root types
        sigma = [0.35, 0.4, 0.4]
        tropN = [0.3, 0.5, 0.9] # strength of tropism
        #RootLif3Time = [50, 40, 40, 30]
        for p in rs.getRootRandomParameter():
            p.dx = 0.5  # adjust resolution
            p.tropismT = pb.TropismType.hydro
            p.tropismN = tropN[p.subType - 1]  # strength of tropism
            p.tropismS = sigma[p.subType - 1]
            #p.rlt = RootLifeTime[i]
        rs.getRootSystemParameter().seedPos = pb.Vector3d(dist * i - (dist/2), dist * j, -3.)  # cm
        
        rs.setSoil(soil_prop) # Set the soil properties before calling initialize
        rs.initialize(False)  # verbose = False
        allRS.append(rs)
        
# Simulate dynamically
simtime = 40  # e.g. 30 or 60 days
dt = 1
N = round(simtime / dt)
for _ in range(0, N):
    for rs in allRS:
        ## in a dynamic soil setting you would need to update the soil properties (soil_prop)
        rs.simulate(dt, True)
    
# Export results as single vtp files (as polylines)
ana = pb.SegmentAnalyser()  # see example 3b
for i, rs in enumerate(allRS):
      vtpname = "results/example_LAB_multiple_hydrotrop_201001a" + str(i) + ".vtp"
      rs.write(vtpname)
      ana.addSegments(rs)  # collect all
    
vp.plot_roots(ana, 'subType', "Multiple Hydro", True)



((vtkRenderingOpenGL2Python.vtkOpenGLActor)0x7f4c94cafc90,
 (vtkRenderingAnnotationPython.vtkScalarBarActor)0x7f4c94cafb40)

In [103]:
# %load example1_LAB200903_multiple_hydrotropism_centerWORKS.py
"""hydrotropism in a middle of a two plants"""
import sys; sys.path.append("../../..")
import plantbox as pb
import vtk_plot as vp

rs = pb.RootSystem()
path = "../../../modelparameter/rootsystem/"
name = ["PS_Pages_mod1bmodA", "PS_Pages_mod1bmodB"]

# Static soil property in a thin layer
maxS = 4  # maximal
minS = 0.1  # minimal
slope = 5  # linear gradient between min and max (cm)
box = pb.SDF_PlantBox(20, 20, 20)  # cm
layer = pb.SDF_RotateTranslate(box, pb.Vector3d(0, 0, -23))
soil_prop = pb.SoilLookUpSDF(layer, maxS, minS, slope)

C = 2  # number of columns
R = 1   # and rows
dist = 15  # distance between the root systems [cm]

# Initializes C*R root systems
allRS = []
for i in range(0, C):
    for j in range(0, R):
        rs = pb.RootSystem()
        rs.readParameters(path + name[i] + ".xml")
        
        # Manually set tropism to hydrotropism for the first six root types
        sigma = [0.4, 0.5, 0.6] * 2
        tropN = [0.5, 0.9, 1.2] * 2 # strength of tropism
        for p in rs.getRootRandomParameter():
            p.dx = 0.25  # adjust resolution
            p.tropismT = pb.TropismType.hydro
            p.tropismN = tropN[p.subType - 1]  # strength of tropism
            p.tropismS = sigma[p.subType - 1]
        rs.getRootSystemParameter().seedPos = pb.Vector3d(dist * i - (dist / 2), dist * j - (dist / 2), -3.)  # cm
        
        rs.setSoil(soil_prop) # Set the soil properties before calling initialize
        rs.initialize(False)  # verbose = False
        allRS.append(rs)
        
# Simulate dynamically
simtime = 30  # e.g. 30 or 60 days
dt = 2
N = round(simtime / dt)
for _ in range(0, N):
    for rs in allRS:
        ## in a dynamic soil setting you would need to update the soil properties (soil_prop)
        rs.simulate(dt, True)
    
# Export results as single vtp files (as polylines)
ana = pb.SegmentAnalyser()  # see example 3b
for i, rs in enumerate(allRS):
      vtpname = "results/example_LAB_multiple_hydrotrop" + str(i) + ".vtp"
#      rs.write(vtpname)
      ana.addSegments(rs)  # collect all
    
vp.plot_roots(ana, 'subType', "Multiple Hydro", True)


((vtkRenderingOpenGL2Python.vtkOpenGLActor)0x7f4c94c33fa0,
 (vtkRenderingAnnotationPython.vtkScalarBarActor)0x7f4c94c33a60)

In [ ]:
N = [0.5, 1., 2.] * 2
print(N)

# odkladiste

In [ ]:
        
# Simulate dynamically
simtime = 25  # e.g. 30 or 60 days
dt = 1
N = round(simtime / dt)
for _ in range(0, N):
    for rs in allRS:
        ## in a dynamic soil setting you would need to update the soil properties (soil_prop)
        rs.simulate(dt, True)

# Simulate
for rs in allRS:
    rs.simulate(simtime, True)  # verbose = False


# Export results as single vtp files (as polylines)
ana = pb.SegmentAnalyser()  # see example 3b
for i, rs in enumerate(allRS):
      vtpname = "results/example_LAB_multiple_hydrotrop" + str(i) + ".vtp"
#      rs.write(vtpname)
      ana.addSegments(rs)  # collect all

# Plot, using vtk
#vp.plot_roots(rs, 'subType', 'oblique', True)
vp.plot_roots(ana, 'subType', "Multiple Hydro", True)

# testujeme kod
# print(pb.SoilLookUpSDF)

In [ ]:
# %load example4a_hydrotropism.py
"""hydrotropism in a thin layer"""
import sys; sys.path.append("../../..")
import plantbox as pb
import vtk_plot as vp

rs = pb.RootSystem()
path = "../../../modelparameter/rootsystem/"
name = "Anagallis_femina_Leitner_2010"
rs.readParameters(path + name + ".xml")

# Manually set tropism to hydrotropism for the first ten root types
sigma = [0.4, 1., 1., 1., 1. ] * 2
for p in rs.getRootRandomParameter():
        p.dx = 0.25  # adjust resolution
        p.tropismT = pb.TropismType.hydro
        p.tropismN = 2  # strength of tropism
        p.tropismS = sigma[p.subType - 1]

# Static soil property in a thin layer
maxS = 0.7  # maximal
minS = 0.1  # minimal
slope = 5  # linear gradient between min and max (cm)
box = pb.SDF_PlantBox(30, 30, 2)  # cm
layer = pb.SDF_RotateTranslate(box, pb.Vector3d(0, 0, -16))
soil_prop = pb.SoilLookUpSDF(layer, maxS, minS, slope)

# Set the soil properties before calling initialize
rs.setSoil(soil_prop)

# Initialize
rs.initialize()

# Simulate
simtime = 100  # e.g. 30 or 60 days
dt = 1
N = round(simtime / dt)
for _ in range(0, N):
    # in a dynamic soil setting you would need to update the soil properties (soil_prop)
    rs.simulate(dt)

# Plot, using vtk
vp.plot_roots(rs, "type", 'oblique', True)


In [ ]:
print('Hello World!')


In [ ]:
import time
time.sleep(3)

In [ ]:
def say_hello(recipient):
    return 'Hello, {}!'.format(recipient)

say_hello('Tim')

In [ ]:
import numpy as np
def square(x):
    return x * x

In [ ]:
x = np.random.randint(1, 10)
y = square(x)
z = 15
print('%d squared is %d a muj test %' % (x, y, z))

In [ ]:
y = 10
print('Is %d squared %d?' % (x, y))

In [ ]:
def greet(name):
    """
    This function greets to
    the person passed in as
    a parameter
    """
    print("Hello, " + name + ". Good morning!")

greet('Paul')

In [ ]:
print(def.__doc__)
    